In [1]:
import pandas as pd
import numpy as np
import sys
import random
import os
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM
from tensorflow.keras.models import Model
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

#### The data

In [3]:
kampala_devices = pd.read_csv('../data/kampala_devices.csv', usecols=['lat', 'long', 'id'])
kampala_devices.head()

,id,lat,long
0,930434,0.360209,32.610756
1,718028,0.307500,32.620600
2,912224,0.346460,32.703280
3,930426,0.365500,32.646800
4,930427,0.268900,32.588000


In [4]:
kampala_df = pd.read_csv('../data/kampala_data.csv', parse_dates=['timestamp'])
kampala_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
1,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
2,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
3,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
4,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26


In [5]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value', 'device_number']
# for i, device_id in enumerate(device_ids):
for i, device_id in kampala_devices.id.iteritems():
    device_df = utils.get_device_data(kampala_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5,device_number
0,453031.0,0.356989,32.613888,10.5477,930434
1,453032.0,0.356989,32.613888,16.4250,930434
2,453033.0,0.356989,32.613888,17.7239,930434
3,453034.0,0.356989,32.613888,16.1533,930434
4,453035.0,0.356989,32.613888,18.0123,930434


In [6]:
latitudes = final_df['latitude'].unique()
longitudes = final_df['longitude'].unique()
device_ids = final_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(35, 35, 34)

In [7]:
final_df = final_df.drop(['device_number'], axis=1)
final_df.head()

,time,latitude,longitude,pm2_5
0,453031.0,0.356989,32.613888,10.5477
1,453032.0,0.356989,32.613888,16.4250
2,453033.0,0.356989,32.613888,17.7239
3,453034.0,0.356989,32.613888,16.1533
4,453035.0,0.356989,32.613888,18.0123


#### Model training and validation

In [8]:
def lstm(X_train, y_train, epochs=1000, optimizer='RMSProp', dropout=0.2):
#     model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.LSTM(50),
#     tf.keras.layers.Dense(1)
# ])
    
    input_layer = Input(shape=(X_train.shape[1],))
    reshaped_input = tf.expand_dims(input_layer, axis=1)
    
    lstm_layer1 = LSTM(units=50, return_sequences=True)(reshaped_input)
    dropout_layer1 = Dropout(dropout)(lstm_layer1)

    lstm_layer2 = LSTM(units=50)(dropout_layer1)
    dropout_layer2 = Dropout(dropout)(lstm_layer2)
    
    output_layer = Dense(units=1)(dropout_layer2)

    model = Model(inputs=input_layer, outputs=output_layer) 

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_split=0.2)
    
#     model = keras.Sequential()
#     model.add(Input(shape=(X_train.shape[1],), name='Input-Layer')) 
#     model.add(LSTM(50, return_sequences=True, input_shape=(window_size, x_train.shape[-1])))
#     model.add(Dropout(rate=dropout))

#     model.add(Bidirectional(LSTM((window_size * 2), return_sequences=True))) 
#     model.add(Dropout(rate=dropout))

#     model.add(Bidirectional(LSTM(window_size, return_sequences=False))) 
#     model.add(Dense(units=1))
    return model

In [9]:
# def scale_data(X):
#     scaler = MinMaxScaler()
#     X_scaled = X.copy()
#     X_scaled[:, 0] = scaler.fit_transform(X[:, 0].reshape(-1, 1)).flatten()
#     return X_scaled

In [10]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = kampala_df[kampala_df.device_number == device_ids[idx]]
#     assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
#     assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
#     assert(len(train_df.longitude.unique()) == len(longitudes)-1)
#     assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
#     X_train_scaled = scale_data(X_train)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
#     X_test_scaled = scale_data(X_test)
    
    model = lstm(X_train, y_train) #, dropout=0.1)
    y_pred = model.predict(X_test)
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape

In [11]:
len(latitudes)

35

In [ ]:
rmse_list, mape_list = [], []
# for i in range(len(latitudes[30:])):
lat_list = list(latitudes)
for lat in latitudes[12:16]:
    i = lat_list.index(lat)
#     print(i)
    try:
        rmse, mape = cross_validation(final_df, i)
        rmse_list.append(rmse)
        mape_list.append(mape)
        print(f'Location {i} successful')
    except Exception as e:
        print(e)
        print(f'Location {i} failed')
#         break

mean_rmse = np.mean(rmse_list)
mean_mape = np.mean(mape_list)
mean_rmse, mean_mape

Epoch 1/1000
1181/1181 [==============================] - 46s 33ms/step - loss: 715.5312 - val_loss: 598.7466
Epoch 2/1000
1181/1181 [==============================] - 38s 32ms/step - loss: 534.9427 - val_loss: 598.4904
Epoch 3/1000
1181/1181 [==============================] - 38s 32ms/step - loss: 534.9205 - val_loss: 598.3521
Epoch 4/1000
1181/1181 [==============================] - 38s 32ms/step - loss: 534.8746 - val_loss: 598.4609
Epoch 5/1000
1181/1181 [==============================] - 38s 32ms/step - loss: 535.4562 - val_loss: 598.3350
Epoch 6/1000
1181/1181 [==============================] - 38s 32ms/step - loss: 535.6093 - val_loss: 598.5438
Epoch 7/1000
1181/1181 [==============================] - 38s 32ms/step - loss: 534.6947 - val_loss: 598.7557
Epoch 8/1000
1181/1181 [==============================] - 37s 31ms/step - loss: 535.4988 - val_loss: 598.3289
Epoch 9/1000
1181/1181 [==============================] - 40s 34ms/step - loss: 534.8266 - val_loss: 598.3517
Epoch 10/1

Epoch 75/1000
1181/1181 [==============================] - 40s 34ms/step - loss: 534.6772 - val_loss: 598.3181
Epoch 76/1000
1181/1181 [==============================] - 41s 34ms/step - loss: 534.0460 - val_loss: 598.9509
Epoch 77/1000
1181/1181 [==============================] - 40s 34ms/step - loss: 534.8583 - val_loss: 598.5651
Epoch 78/1000
1181/1181 [==============================] - 41s 35ms/step - loss: 534.3306 - val_loss: 598.3776
Epoch 79/1000
1181/1181 [==============================] - 40s 34ms/step - loss: 533.9421 - val_loss: 598.7904
Epoch 80/1000
1181/1181 [==============================] - 41s 35ms/step - loss: 534.2917 - val_loss: 598.4512
Epoch 81/1000
1181/1181 [==============================] - 40s 34ms/step - loss: 533.8861 - val_loss: 598.4625
Epoch 82/1000
1181/1181 [==============================] - 38s 33ms/step - loss: 533.8161 - val_loss: 598.3138
Epoch 83/1000
1181/1181 [==============================] - 40s 34ms/step - loss: 534.0643 - val_loss: 598.3561
E

1181/1181 [==============================] - 41s 35ms/step - loss: 534.1788 - val_loss: 598.3160
Epoch 149/1000
1181/1181 [==============================] - 40s 34ms/step - loss: 533.9182 - val_loss: 598.5613
Epoch 150/1000
1181/1181 [==============================] - 42s 36ms/step - loss: 533.2211 - val_loss: 598.8284
Epoch 151/1000
1181/1181 [==============================] - 41s 35ms/step - loss: 533.4016 - val_loss: 599.4177
Epoch 152/1000
1181/1181 [==============================] - 41s 35ms/step - loss: 533.4695 - val_loss: 599.2836
Epoch 153/1000
1181/1181 [==============================] - 40s 34ms/step - loss: 533.0553 - val_loss: 598.8934
Epoch 154/1000
1181/1181 [==============================] - 41s 35ms/step - loss: 533.1453 - val_loss: 598.4077
Epoch 155/1000
1181/1181 [==============================] - 40s 34ms/step - loss: 532.9868 - val_loss: 598.3585
Epoch 156/1000
1181/1181 [==============================] - 42s 35ms/step - loss: 533.3431 - val_loss: 598.5370
Epoch 1

1181/1181 [==============================] - 41s 35ms/step - loss: 533.2397 - val_loss: 598.8609
Epoch 222/1000
1181/1181 [==============================] - 42s 35ms/step - loss: 533.4875 - val_loss: 600.2313
Epoch 223/1000
1181/1181 [==============================] - 41s 34ms/step - loss: 532.9349 - val_loss: 599.5643
Epoch 224/1000
1181/1181 [==============================] - 41s 35ms/step - loss: 532.9916 - val_loss: 598.4893
Epoch 225/1000
1181/1181 [==============================] - 40s 34ms/step - loss: 532.5566 - val_loss: 602.3818
Epoch 226/1000
1181/1181 [==============================] - 42s 35ms/step - loss: 533.1873 - val_loss: 598.9850
Epoch 227/1000
1181/1181 [==============================] - 39s 33ms/step - loss: 533.2402 - val_loss: 598.5140
Epoch 228/1000
1181/1181 [==============================] - 39s 33ms/step - loss: 532.5569 - val_loss: 599.6579
Epoch 229/1000
1181/1181 [==============================] - 40s 34ms/step - loss: 532.7501 - val_loss: 600.3177
Epoch 2

In [ ]:
rmse_list

In [ ]:
mape_list